In [1]:
from functorch import vmap
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import *
import pathlib
import pickle
import einops
import math
from torchvision.models import resnet50, ResNet50_Weights, resnet101, ResNet101_Weights
import torchvision        


img_encoder = torchvision.models.resnet101(
            weights=ResNet101_Weights.IMAGENET1K_V2
        )  # shape out = 512 for resnet 18

# hook the feature extractor 

/home/ptthang/gaze_sample/env_torchism/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def image_encoder_forward(x):
    # resnet forward
    x = img_encoder.conv1(x)
    x = img_encoder.bn1(x)
    x = img_encoder.relu(x)
    x = img_encoder.maxpool(x)

    x = img_encoder.layer1(x)
    x = img_encoder.layer2(x)
    x = img_encoder.layer3(x)
    x = img_encoder.layer4(x)
    return x

In [11]:
image_encoder_forward(torch.rand(1,3,224,224)).shape

torch.Size([1, 2048, 7, 7])

In [12]:
img_encoder.

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 